# **Homework 5: Basic Block Recursivity**

## **ECON8582: Macro-Labour**

#### *Conor Bayliss*

In [1]:
using LinearAlgebra, QuantEcon, Random, CairoMakie

#### **Calibration**

In [2]:
pars = (;β = 0.996, # discount factor
        ρ = 0.98, # autocorrelation of productivity process
        σe = 0.01, # standard deviation of productivity process
        nz = 15, # number of points in the productivity grid
        γ = 0.6, # coefficient of job finding rate
        κ = 1.89, # vacancy posting cost
        δ = 0.012, # separation rate
        σ = 0, # CRRA parameter
        α = 1, # search cost parameter
        χ = 2, # search cost exponent
        a = 1/3, # λ(s) = s^a job posting opportunities
        w_lb = .1, # lower bound of wage grid
        w_ub = 1, # upper bound of wage grid
        Δw = 0.025, # wage grid step
        b = 0.54, # benefit parameter
        s_lb = 0.2, # lower bound of search cost grid
        s_ub = 0.4, # upper bound of search cost grid
        Δs = 0.02, # search cost grid step
        toler = 1e-6, # tolerance for convergence
        maxiter = 1000) # maximum number of iterations

(β = 0.996, ρ = 0.98, σe = 0.01, nz = 15, γ = 0.6, κ = 1.89, δ = 0.012, σ = 0, α = 1, χ = 2, a = 0.3333333333333333, w_lb = 0.1, w_ub = 1, Δw = 0.025, b = 0.54, s_lb = 0.2, s_ub = 0.4, Δs = 0.02, toler = 1.0e-6, maxiter = 1000)

##### **Income process**

In [3]:
function income_process(p)
    (;ρ, σe, nz) = p
    mc = rouwenhorst(nz, ρ, σe)
    z_grid = exp.(mc.state_values)
    Π = mc.p
    return z_grid, Π 
end

test_z_grid, test_Π = income_process(pars)

([0.8285936952614785, 0.8511519660876842, 0.8743243805956225, 0.8981276587043299, 0.9225789755287533, 0.9476959737723538, 0.9734967764570943, 1.0, 1.027224767645724, 1.0551907232648117, 1.0839180455276196, 1.1134274624641165, 1.14374026642007, 1.174878329420415, 1.2068641189508822], [0.8687458127689781 0.1228529432198556 … 1.386000000000016e-25 1.0000000000000126e-28; 0.008775210229989686 0.8698981131022091 … 1.2741400000000137e-23 9.900000000000117e-27; … ; 9.900000000000117e-27 1.2741400000000137e-23 … 0.8698981131022091 0.008775210229989686; 1.0000000000000126e-28 1.386000000000016e-25 … 0.1228529432198556 0.8687458127689781])

##### **Wage grid**

In [4]:
function wage_grid(pars)
    (;w_lb, w_ub, Δw) = pars
    w_grid = range(w_lb, w_ub, step = Δw)
    return w_grid
end

test_w_grid = wage_grid(pars)

0.1:0.025:1.0

##### **Search grid**

In [5]:
function search_cost_grid(pars)
    (;s_lb, s_ub, Δs) = pars
    s_grid = range(s_lb, s_ub, step = Δs)
    return s_grid
end

test_s_grid = search_cost_grid(pars)

0.2:0.02:0.4

##### **Job finding rate**

In [6]:
function job_finding_rate(θ_grid, pars)
    (;γ) = pars
    p = θ_grid.*(1. .+ θ_grid.^γ).^(-1/γ)
    return p
end

test_find_rate = job_finding_rate(test_s_grid, pars)

11-element Vector{Float64}:
 0.11681940254944001
 0.12509945901524733
 0.13304036654489015
 0.14067162398429822
 0.14801861621348178
 0.1551034048002631
 0.16194532737867068
 0.16856146074207967
 0.17496698456267964
 0.18117547118705724
 0.18719911946843884

##### **Job posting opportunities**

In [7]:
function job_opp(pars)
    (;a) = pars
    temp_s_grid = search_cost_grid(pars)
    job_opp = temp_s_grid.^a
    return job_opp
end

test_job_opp = job_opp(pars)

11-element Vector{Float64}:
 0.5848035476425733
 0.6036810736797686
 0.6214465011907718
 0.6382504298859908
 0.654213262037718
 0.6694329500821695
 0.6839903786706788
 0.6979532046908887
 0.7113786608980126
 0.7243156443441741
 0.7368062997280773

##### **Benefits grid**

In [8]:
function benefits_grid(pars)
    (;b) = pars
    temp_w_grid = wage_grid(pars)
    benefits = b .* temp_w_grid
    return benefits
end

test_benefits = benefits_grid(pars)

0.054000000000000006:0.013500000000000002:0.54

##### **Utility**

In [9]:
function utility(c,pars)
    (;σ) = pars
    
    if σ == 1
        u = log
    else
        u = x -> (x^(1-σ) - 1)/(1-σ)
    end

    return u(c)
end

test_c = utility(2,pars)

1.0

##### **Job destruction**

In [10]:
function job_dest(z,w,pars)
    (;δ) = pars
    if z >= w
        return δ
    else
        return 1
    end
end

@show test_z_grid[5]
@show test_w_grid[34]
test_job_dest = job_dest(test_z_grid[5],test_w_grid[34],pars)

test_z_grid[5] = 0.9225789755287533
test_w_grid[34] = 0.925


1

##### **Search costs**

In [11]:
function search_costs(s,pars)
    (;α, χ) = pars
    return α .* (s.^χ)
end

test_search_costs = search_costs(test_s_grid,pars)

11-element Vector{Float64}:
 0.04000000000000001
 0.0484
 0.0576
 0.06760000000000001
 0.07840000000000001
 0.09
 0.1024
 0.11560000000000002
 0.1296
 0.1444
 0.16000000000000003

##### **Initial guess for $\mathbf{J}$**

In [12]:
function initial_J(pars)
    (;nz,w_ub,w_lb,Δw) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    J = zeros(nw,nz)
    return J
end

test_J = initial_J(pars)

37×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0 

#### **Value function iteration on J**

In [18]:
function iterate(pars)
    (;nz, w_ub, w_lb, Δw, a, b, γ, κ, δ, α, χ, β, toler, maxiter) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    J_init = initial_J(pars)
    J_new = copy(J_init)
    error = 1
    iter = 1
    z_grid, Π = income_process(pars)
    w_grid = wage_grid(pars)
    EV = zeros(nw,nz)
    while iter < maxiter
        J_new = copy(J_init)
        for i in 1:nw
            for j in 1:nz
                EV[i,j] = 0
                for jp in 1:nz
                    EV[i,j] += J_init[i,jp] * Π[j,jp]
                end
            end
            J_init[i,:] = z_grid .- w_grid[i] .+ (β .* (1 .- job_dest.(z_grid,w_grid[i],Ref(pars)))) .* EV[i,:]
        end
        error = maximum(abs.(J_new - J_init))
        J_new = copy(J_init)
        if error < toler
            println("--------------------")
            println("Converged in $iter iterations")
            println("Error: $error")
            println("--------------------")
            break
        end
        if iter == maxiter
            println("Maximum number of iterations reached")
        end
        if iter % 1 == 0
            println("--------------------")
            println("Iteration: $iter")
            println("Error: $error")
            println("--------------------")
        end
        iter += 1
    end
    return J_new
end

J_out = iterate(pars)
    

37×15 Matrix{Float64}:
 51.5029     52.1771    52.8629       …  60.4167   61.2475   62.093
 49.9357     50.6099    51.2957          58.8495   59.6803   60.5258
 48.3685     49.0427    49.7285          57.2823   58.1131   58.9586
 46.8013     47.4755    48.1613          55.7151   56.5459   57.3914
 45.2341     45.9083    46.5941          54.1479   54.9787   55.8242
 43.6669     44.3411    45.0269       …  52.5807   53.4115   54.257
 42.0997     42.7739    43.4597          51.0135   51.8443   52.6898
 40.5325     41.2067    41.8925          49.4463   50.2771   51.1226
 38.9653     39.6395    40.3253          47.8791   48.7099   49.5554
 37.3981     38.0723    38.7581          46.3119   47.1427   47.9882
  ⋮                                   ⋱                      
  7.62132     8.29549    8.98132         16.5351   17.3659   18.2114
  6.05412     6.72829    7.41412         14.9679   15.7987   16.6442
 -0.0214063   4.85121    5.7971       …  13.4004   14.2312   15.0768
 -0.0464063  -0.0238